Business Problem
-

Model ML pada studi kasus travel insurance ini nantinya bisa digunakan oleh perusahaan untuk memprediksi, 
berdasarkan fitur-fitur, pemegang polis seperti apa yang akan mengajukan klaim. Hal ini penting karena dari prediksi tersebut perusahaan dapat mengelelola risiko terhadap keuangan perusahaan seperti penentuan penyediaan budget untuk asuransi-asuransi yang diklaim, penentuan pricing produk-produk asuransi, pendeteksian fraud, dan lain-lain sehingga perusahaan akan lebih sustain.

Suatu studi kasus, kolaborasi antara perusahaan asuransi Brasil dan Actdigital.com menjelaskan bagaimana perusahaan asuransi tersebut menggunakan analisis prediktif untuk mencegah klaim yang tidak tepat dan menghemat BRL 16,6 juta (~USD 3,3 juta) di tahun pertama. Sedikit penurunan klaim yang terlewat dapat menghemat bahkan jutaan dolar dalam pembayaran. Dapat dibayangkan jika yang terjadi sebaliknya bahwa perusahaan mengizinkan klaim yang tidak tepat, tentu perusahaan akan mengalami kerugian dan menjadi masalah besar terhadap keuangan mereka.

Selain menghindari kerugian, Pricing polis juga terbantu dengan mampu menentukannya berdasarkan profil pelanggan yang terperinci secara real-time. Contohnya segmen pelanggan berisiko tinggi (seperti perjalanan jauh dengan harga premium) yang dapat dikenakan premi lebih tinggi atau memerlukan underwriting tambahan. 

Pemodelan ML yang tepat dapat menghasilkan biaya yang lebih rendah dan produk yang lebih inovatif sehingga meningkatkan profitabilitas perusahaan.

Data Understanding
-

In [79]:
import pandas as pd

df = pd.read_csv('data_travel_insurance.csv')
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age,Claim
0,C2B,Airlines,Online,Annual Silver Plan,F,365,SINGAPORE,216.0,54.0,57,No
1,EPX,Travel Agency,Online,Cancellation Plan,NaN,4,MALAYSIA,10.0,0.0,33,No
2,JZI,Airlines,Online,Basic Plan,M,19,INDIA,22.0,7.7,26,No
3,EPX,Travel Agency,Online,2 way Comprehensive Plan,NaN,20,UNITED STATES,112.0,0.0,59,No
4,C2B,Airlines,Online,Bronze Plan,M,8,SINGAPORE,16.0,4.0,28,No


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44328 entries, 0 to 44327
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                44328 non-null  object 
 1   Agency Type           44328 non-null  object 
 2   Distribution Channel  44328 non-null  object 
 3   Product Name          44328 non-null  object 
 4   Gender                12681 non-null  object 
 5   Duration              44328 non-null  int64  
 6   Destination           44328 non-null  object 
 7   Net Sales             44328 non-null  float64
 8   Commision (in value)  44328 non-null  float64
 9   Age                   44328 non-null  int64  
 10  Claim                 44328 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 3.7+ MB


Terdapat 4 data numerikal dan 7 data kategorikal yang terdiri dari:

Data Numerikal
- Duration : Lama perjalanan
- Net Sales : Jumlah sales dari polis asuransi
- Commision (in value) : Komisi yang didapatkan agency
- Age : Usia pemegang polis asuransi

Data Kategorikal
- Agency : Nama agensi
- Agency Type : Tipe agensi 
- Distribution Channel : Channel yang digunakan untuk menjual asuransi
- Product Name : Nama produk asuransi
- Gender : Jenis kelamin pemegang polis
- Destination : Tujuan perjalanan
- Claim : Status klaim apakah diajukan atau tidak

Data tersebut baik untuk mentraining suatu model klasifikasi untuk memprediksi claim

Data Cleaning
-

In [81]:
df.duplicated().sum()

4667

In [82]:
df.drop_duplicates(inplace=True)

Data duplikat dihapus agar data lebih valid dan meningkatkan keandalan dari hasil pemodelan

In [84]:
df.isnull().sum().sort_values(ascending=False)

Gender                  27667
Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Age                         0
Claim                       0
dtype: int64

In [85]:
misval=df['Gender'].isna().sum()
totalrows=len(df)
percentmis=(misval/totalrows)*100
print(f"{percentmis:.2f}%")

69.76%


Persentase rows dengan missing value pada gender mencapai lebih dari 60% sehingga handling tidak dilakukan dengan
mendrop

In [86]:
df['Gender'].value_counts(normalize=True)

Gender
M    0.50642
F    0.49358
Name: proportion, dtype: float64

Karena proporsi gender male dan female hampir sama, maka dilakukan impute dengan random sampling untuk
mempertahankan angka proporsi

In [87]:
import numpy as np

# Calculate proportions
gender_probs = df['Gender'].value_counts(normalize=True)

# Randomly assign missing values based on observed distribution
df.loc[df['Gender'].isna(), 'Gender'] = np.random.choice(
    gender_probs.index,
    size=df['Gender'].isna().sum(),
    p=gender_probs.values
)

In [88]:
df['Gender'].value_counts(normalize=True)

Gender
M    0.505812
F    0.494188
Name: proportion, dtype: float64

In [89]:
df.describe()

,Duration,Net Sales,Commision (in value),Age
count,39661.000000,39661.000000,39661.000000,39661.000000
mean,52.397822,42.342794,10.442622,39.930284
std,113.542824,50.025244,20.355921,13.526346
min,-1.000000,-357.500000,0.000000,0.000000
25%,11.000000,19.000000,0.000000,34.000000
50%,25.000000,29.000000,0.000000,36.000000
75%,57.000000,50.000000,11.880000,45.000000
max,4881.000000,810.000000,283.500000,118.000000


Features yang akan dihandling
- Duration : Teradapat nilai minimum -1 dan maximum 4881 hari (Syarat pemegang polis tidak boleh > 180 hari)
- Net Sales : Terdapat nilai < 0
- Age : Terdapat value diatas 100

In [90]:
# Handling duration

df = df[(df['Duration'] > 0) & (df['Duration'] <= 180)]

In [91]:
# Handling net sales

df = df[df["Net Sales"] > 0]

In [92]:
# Handling Age > 100

df = df[df["Age"] < 100]

Feature Engineering
-

In [93]:
df['Claim'] = df['Claim'].map({'No': 0, 'Yes': 1})

df['Duration_Group'] = pd.cut(df['Duration'], 
                             bins=[0, 7, 14, 30, 90, 365],
                             labels=['0-7', '8-14', '15-30', '31-90', '91-365'])

df['Age_Group'] = pd.cut(df['Age'],
                        bins=[0, 18, 30, 50, 70, 100],
                        labels=['0-18', '19-30', '31-50', '51-70', '71-100'])

Modelling
-

In [94]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Separate features and target
X = df.drop('Claim', axis=1)
y = df['Claim']

# Define categorical and numerical features
categorical_features = ['Agency', 'Agency Type', 'Distribution Channel', 
                       'Product Name', 'Gender', 'Destination', 
                       'Duration_Group', 'Age_Group']
numerical_features = ['Duration', 'Net Sales', 'Commision (in value)', 
                     'Age']

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Create pipeline with preprocessing and model
baseline_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',
        random_state=42,
        max_iter=1000,          # Increased from default 100
        solver='lbfgs',         # Default solver (good for most cases)
        tol=1e-3                # Slightly relaxed convergence tolerance
    ))
])

# Train and evaluate
baseline_model.fit(X_train, y_train)
y_pred = baseline_model.predict(X_test)

print("Baseline Model Performance:")
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, baseline_model.predict_proba(X_test)[:, 1]))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Baseline Model Performance:
              precision    recall  f1-score   support

           0       0.99      0.77      0.87      6967
           1       0.04      0.69      0.08       104

    accuracy                           0.77      7071
   macro avg       0.52      0.73      0.48      7071
weighted avg       0.98      0.77      0.86      7071

ROC AUC: 0.7803270362478055

Confusion Matrix:
[[5377 1590]
 [  32   72]]


In [96]:
df["Claim"].value_counts(normalize=True)

Claim
0    0.985292
1    0.014708
Name: proportion, dtype: float64

Terlihat bahwa proporsi claim (Yes=1) jauh
lebih sedikit (imbalance). Berdasarkan business problem bahwa suatu perusahaan travel insurance bisa sangat 
dirugikan jika melakukan claim dengan tidak tepat maka model yang nantinya akan dibuat difokuskan untuk mendapat
recall rate yang baik untuk dapat mendeteksi positive class (claim = 'Yes'/1)

In [55]:
# Try different class weights
weights = [{0:1, 1:10}, {0:1, 1:25}, {0:1, 1:50}, {0:1, 1:75}]

for weight in weights:
    logreg_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',
        max_iter=2000,  # High enough to ensure convergence
        solver='saga',  # Most robust solver
        tol=1e-3,       # Slightly looser tolerance
        random_state=42
    ))
    ])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\nWeights {weight}:")
    print(classification_report(y_test, y_pred))

C:\Users\marka\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Weights {0: 1, 1: 10}:
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      6967
           1       0.04      0.72      0.08       104

    accuracy                           0.74      7071
   macro avg       0.52      0.73      0.46      7071
weighted avg       0.98      0.74      0.84      7071



C:\Users\marka\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Weights {0: 1, 1: 25}:
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      6967
           1       0.04      0.72      0.08       104

    accuracy                           0.74      7071
   macro avg       0.52      0.73      0.46      7071
weighted avg       0.98      0.74      0.84      7071



C:\Users\marka\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Weights {0: 1, 1: 50}:
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      6967
           1       0.04      0.72      0.08       104

    accuracy                           0.74      7071
   macro avg       0.52      0.73      0.46      7071
weighted avg       0.98      0.74      0.84      7071


Weights {0: 1, 1: 75}:
              precision    recall  f1-score   support

           0       0.99      0.75      0.85      6967
           1       0.04      0.72      0.08       104

    accuracy                           0.74      7071
   macro avg       0.52      0.73      0.46      7071
weighted avg       0.98      0.74      0.84      7071



C:\Users\marka\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Untuk menghandling data imbalance maka dilakukan teknik different class weight dimana hasilnya recall lebih
tinggi dari baseline model sebelumnya

Kesimpulan
-

Logistic Regression setelah dilakukan pembobotan dengan class yang berbeda dapat meningkatkan recall

Rekomendasi
-

* **Kurangi False Positives:** Implementasikan cost-sensitive learning atau threshold tuning untuk menyeimbangkan recall dan presisi, mengurangi beban operasional akibat penanganan kasus non-klaim yang salah diprediksi.

* **Tingkatkan Kualitas Data:** Lakukan pembersihan data lebih lanjut (misalnya, tangani usia tidak realistis seperti 0 atau 118) dan tambahkan fitur baru (contoh: indeks risiko destinasi) untuk meningkatkan akurasi prediksi.

* **Validasi dan Generalisasi:** Uji model pada data eksternal atau skenario baru (misalnya, musim libur atau krisis) untuk memastikan performa stabil, menghindari kerugian akibat prediksi yang tidak akurat.

* **Strategi Pelanggan:** Gunakan hasil prediksi untuk komunikasi proaktif dengan pelanggan berisiko tinggi, seperti menawarkan paket asuransi tambahan atau edukasi risiko, meningkatkan kepuasan dan retensi pelanggan.

In [77]:
import joblib

# 1. Save the trained model pipeline
joblib.dump(model, 'travel_insurance_model.pkl')

['travel_insurance_model.pkl']